In [ ]:
import psycopg2
import pandas as pd

In [ ]:
# Connect to the database
conn = psycopg2.connect(
    dbname="school_db",
    user="uthkarsh",
    password="Ruthwik081@",
    host="localhost",
    port="5432"
)

In [ ]:
# Query data
df = pd.read_sql_query("SELECT * FROM students", conn)